In [16]:
import numpy as np
import pandas as pd
from scipy.optimize import fmin, minimize, fminbound

from sklearn.metrics import mean_squared_error, mean_absolute_error

import requests
import json

In [17]:
pd.set_option("display.max_colwidth", 0)

In [18]:
# x11 = 1; x12 = 2; x13 = 3
# x1 = [x11*4+x12*4+x13*8, x11, x12, x13]
# x21 = 4; x22 = 3; x23 = 2
# x2 = [x21*4+x22*4+x23*8, x21, x22, x23]

In [19]:
# x = np.array([x1, x2])

In [20]:
# Daily goal
y1 = 136; y2 = 95; y3 = 170
y = np.array([y1, y2, y3, y1*4+y2*8+y3*4])

In [21]:
y

array([ 136,   95,  170, 1984])

In [22]:
# Relative MSE
def mse(A, B):
    return (((A - B)/A) ** 2).mean(axis=0)

In [23]:
# init = [2]*len(x)
# bounds = tuple((0, None) for x in init)

In [24]:
# result = minimize(f, init, method='SLSQP', bounds=bounds)
# result

In [25]:
# print y
# print np.dot(result.x, x)

In [26]:
apikey = '3jklPYmeuoykAvPVFSkTTXycJlxupddyBKtItMzg'

In [27]:
# [n['nutrient'] for n in picked[0]['nutrients']]

In [28]:
# payload = {'api_key': apikey,
#            'fg': ['0500', '1300', '1500', '0900', '1700', '1600', '1200', '1000', '0500', '1100'],
#            'nutrients': [208, 203, 205, 204],
#            'max': 1500,
#           }
# r = requests.get('http://api.nal.usda.gov/ndb/nutrients/', params=payload)
# foods = json.loads(r.text)['report']['foods']

In [29]:
foods = {}
for fg in ['0500', '1300', '1500', '0900', '1700', '1600', '1200', '1000', '0500', '1100']:
    print fg,
    payload = {'api_key': apikey,
               'fg': fg,
               'nutrients': [208, 203, 205, 204],
               'max': 1500,
               'subset': 1
              }
    r = requests.get('http://api.nal.usda.gov/ndb/nutrients/', params=payload)
    report = json.loads(r.text)['report']
    foods[str(report['groups'][0]['description'])] = report['foods']

0500 1300 1500 0900 1700 1600 1200 1000 0500 1100


In [30]:
foods.keys()

['Finfish and Shellfish Products',
 'Legumes and Legume Products',
 'Vegetables and Vegetable Products',
 'Beef Products',
 'Lamb, Veal, and Game Products',
 'Pork Products',
 'Nut and Seed Products',
 'Fruits and Fruit Juices',
 'Poultry Products']

In [31]:
picked = []
groups = []
for k, v in foods.items():
    if k in ['Vegetables and Vegetable Products']:
        size = 3
    else:
        size = 1
    if k in ['Finfish and Shellfish Products', 'Beef Products', 'Lamb, Veal, and Game Products', 'Pork Products', 'Poultry Products', 'Vegetables and Vegetable Products']:
        v = [i for i in v if 'raw' in i['name']]
    p = np.random.choice(v, size, False)
    picked.append(p)
    for i in range(size):
        groups.append(k)
picked = np.hstack(picked)

In [90]:
def refresh(idx):
    for i, k in enumerate(groups):
        v = foods[k]
        if i == idx:
            if k in ['Finfish and Shellfish Products', 'Beef Products', 'Lamb, Veal, and Game Products', 'Pork Products', 'Poultry Products', 'Vegetables and Vegetable Products']:
                v = [i for i in v if 'raw' in i['name']]
            p = np.random.choice(v, 1, False)
          
            print 'Replaced {} with {}'.format(picked[idx]['name'], p[0]['name'])
            picked[idx] = p[0]
    return [p['name'] for p in picked]

In [99]:
names = refresh(4)

Replaced Cress, garden, raw with Peas, edible-podded, raw


In [100]:
x = np.array([[float(n[0]['gm']), float(n[1]['gm']), float(n[2]['gm']), float(n[3]['gm'])] 
              for n in [p['nutrients'] for p in picked]])
init = [2]*len(x)
bounds = tuple((0, None) for x in init)

In [105]:
# function to minimize
n_days = 5
def f(a):
    pred = np.dot(a, x)
    return mse(y*n_days, pred) #mean_absolute_error(y, pred)

result = minimize(f, init, method='SLSQP', bounds=bounds)
df = pd.DataFrame({'name': names, 'amount, g': np.round(result.x*100), 
                   'amount, oz': np.round(result.x*100*0.035274),
                   'amount, lbs': np.around(result.x*100*0.00220462, 2),
                   'group': groups})

In [106]:
print y
print np.round(np.dot(result.x, x)/5)
df
# print np.dot(np.where(result.x>0.1, result.x, 0), x)

[ 136   95  170 1984]
[  136.    95.   170.  1982.]


,"amount, g","amount, lbs","amount, oz",group,name
0,594,1.31,21,Finfish and Shellfish Products,"Fish, haddock, raw"
1,880,1.94,31,Legumes and Legume Products,"LOMA LINDA Linketts, canned, unprepared"
2,721,1.59,25,Vegetables and Vegetable Products,"Arugula, raw"
3,1071,2.36,38,Vegetables and Vegetable Products,"Cauliflower, green, raw"
4,1285,2.83,45,Vegetables and Vegetable Products,"Peas, edible-podded, raw"
5,622,1.37,22,Beef Products,"Beef, ribeye petite roast/filet, boneless, separable lean only, trimmed to 0"" fat, select, raw"
6,202,0.44,7,"Lamb, Veal, and Game Products","Lamb, Australian, imported, fresh, foreshank, separable lean and fat, trimmed to 1/8"" fat, raw"
7,227,0.50,8,Pork Products,"Pork, fresh, shoulder, (Boston butt), blade (steaks), separable lean and fat,with added solution, raw"
8,463,1.02,16,Nut and Seed Products,"Nuts, hazelnuts or filberts"
9,2825,6.23,100,Fruits and Fruit Juices,"Cherries, sour, red, canned, light syrup pack, solids and liquids"
